In [2]:
import json
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import Sort, reviews, app

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

app_packages = [
    'com.google.android.apps.meetings',
    'com.instagram.android',
    'com.microsoft.teams',
    'com.netflix.mediaclient',
    'com.zhiliaoapp.musically',
    'com.whatsapp',
    'us.zoom.videomeetings'
]

app_infos = []

for ap in tqdm(app_packages):
  info = app(ap, lang='en', country='us')
  del info['comments']
  app_infos.append(info)    

app_infos_df = pd.DataFrame(app_infos)
app_infos_df.to_csv('apps.csv', index=None, header=True)

app_reviews = []

for ap in tqdm(app_packages):
  for score in list(range(1, 6)):
    for sort_order in [Sort.NEWEST]:
      rvs, _ = reviews(
        ap,
        lang='en',
        country='us',
        sort=sort_order,
        count= 2000,
        filter_score_with=score
      )
      for r in rvs:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
      app_reviews.extend(rvs)

app_reviews_df = pd.DataFrame(app_reviews)
app_reviews_df.to_csv('reviews.csv', index=None, header=True)

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [12:25<00:00, 106.57s/it]


In [10]:
import pandas as pd
data = pd.read_csv("reviews_google_meet.csv")
data_text = data[['content']]
data_text['index'] = data_text.index
documents = data_text

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer
import numpy as np

def lemmatize_only(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_only(token))
    return result

doc_sample = documents[documents['index'] == 4310].values[0][0]

processed_docs = documents['content'].map(preprocess)
dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=5000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

print('\nBag of Words Model:\n')
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

print('\nTF-IDF Model:\n')
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))


Bag of Words Model:

Topic: 0 
Words: 0.304*"good" + 0.037*"meet" + 0.034*"work" + 0.028*"nice" + 0.024*"data" + 0.021*"awesome" + 0.020*"time" + 0.019*"worst" + 0.017*"problem" + 0.013*"take"
Topic: 1 
Words: 0.058*"good" + 0.040*"meet" + 0.031*"video" + 0.030*"voice" + 0.029*"chat" + 0.024*"problem" + 0.024*"nice" + 0.020*"join" + 0.019*"clear" + 0.018*"time"
Topic: 2 
Words: 0.077*"video" + 0.045*"quality" + 0.041*"like" + 0.032*"class" + 0.031*"meet" + 0.030*"audio" + 0.027*"great" + 0.020*"rate" + 0.018*"best" + 0.016*"google"
Topic: 3 
Words: 0.086*"meet" + 0.041*"google" + 0.036*"work" + 0.027*"nice" + 0.027*"time" + 0.026*"good" + 0.023*"join" + 0.019*"update" + 0.019*"better" + 0.019*"experience"
Topic: 4 
Words: 0.072*"meet" + 0.064*"class" + 0.044*"good" + 0.041*"online" + 0.027*"join" + 0.025*"google" + 0.024*"problem" + 0.016*"helpful" + 0.016*"thank" + 0.015*"help"
Topic: 5 
Words: 0.079*"meet" + 0.062*"best" + 0.050*"screen" + 0.026*"record" + 0.021*"good" + 0.018*"feat